In [ ]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

     |████████████████████████████████| 344 kB 5.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595725 sha256=080a87b0fe2028b386bfcec33e13f7cf032a84edef40ca452a318c375802bab0
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 47.3 MB 1.6 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 1.1 MB 5.3 MB/s 
     |████████████████████████████████| 2.5 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 31.7 MB/s 
     |████████████████████████████████| 895 kB 45.6 MB/s 


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-hohqpqp6/kobert-tokenizer_9abb82ad9bea414d9b0e189150aa06cf
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-hohqpqp6/kobert-tokenizer_9abb82ad9bea414d9b0e189150aa06cf
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=fed43d3e31a493642f088a6936baef7719ded7b128bed296f03f061955d39861
  Stored in directory: /tmp/pip-ephem-wheel-cache-llpb0b6l/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-w7eiebo0
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-w7eiebo0
     |████████████████████████████████| 132 kB 5.4 MB/s 
     |████████████████████████████████| 4.5 MB 34.9 MB/s 
     |████████████████████████████████| 8.6 MB 11.9 MB/s 
     |████████████████████████████████| 79 kB 6.7 MB/s 
     |████████████████████████████████| 127 kB 35.3 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=10d9c83759619933861c15e81d5f58f8336aa92c22d531a49f76927c038902e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-3a2nalhh/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/Othercomputers/drive/drive/findata.csv')

In [ ]:
df = df.iloc[: ,1:]

In [ ]:
df.head()

,hotel,star,review,date,real_date,length
0,그랜드 하얏트 서울,5,최고의 주말 데이트를 했습니다,1일 전,2022. 03. 14,16
1,그랜드 하얏트 서울,5,조식 맛집,2일 전,2022. 03. 13,5
2,그랜드 하얏트 서울,5,야경이 너무너무 멋졌어요,2022. 03. 07,2022. 03. 07,13
3,그랜드 하얏트 서울,4,체크인 시간 전에 도착 시 대기 할 공간이 미흡했던 것같고 시설 이용할 수 있는게 ...,2022. 03. 07,2022. 03. 07,60
4,그랜드 하얏트 서울,5,직원분이 너무너무 친절하셔서 기분 좋게 잘 다녀왔습니다,2022. 03. 07,2022. 03. 07,30


In [ ]:
def rating_to_label(rating):
    if rating > 4:
        return 1
    else:
        return 0
    
df['label'] = df['star'].apply(lambda x: rating_to_label(x))


In [ ]:
df["label"].value_counts()

1    17723
0     5198
Name: label, dtype: int64

In [ ]:
df = df[['review','label']]

In [ ]:
df.head()

,review,label
0,최고의 주말 데이트를 했습니다,1
1,조식 맛집,1
2,야경이 너무너무 멋졌어요,1
3,체크인 시간 전에 도착 시 대기 할 공간이 미흡했던 것같고 시설 이용할 수 있는게 ...,0
4,직원분이 너무너무 친절하셔서 기분 좋게 잘 다녀왔습니다,1


In [ ]:
df.to_csv('findata2.csv')

In [ ]:
dataset = nlp.data.TSVDataset('/content/drive/Othercomputers/drive/drive/findata2.tsv', field_indices=[1,2], num_discard_samples=1)

In [ ]:
dataset[5]

['걸어가긴 힘든데 넘 조아요', '1']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22921 entries, 0 to 22920
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  22629 non-null  object
 1   label   22921 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 358.3+ KB


In [ ]:
data_list = []
for q, label in zip(df['review'], df['label'])  :
    data = []
    data.append(q)
    data.append(label)

    data_list.append(data)

In [ ]:
print(data_list[2000])
print(data_list[5000])

['오션뷰가 나쁘진 않은데 트인 느낌은 아니에요수영장도 생각보단 작네요', 0]
['가격대비 최고의 호텔인듯', 1]


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(dataset, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

17190
5731


In [ ]:
dataset_train[500]

['신분확인을 카드 2개로 하는 이유를 모르겠음', '0']

리뷰데이터 토큰화, 정수인코딩

In [ ]:
!pip install --no-cache-dir transformers sentencepiece

In [ ]:
result = kobert_tokenizer.tokenize(data_list[3][0])
print(result)
kobert_vocab = kobert_tokenizer.get_vocab()
print([kobert_tokenizer.encode(token) for token in result])

['▁체크', '인', '▁시간', '▁전에', '▁도착', '▁시', '▁대기', '▁할', '▁공간', '이', '▁미', '흡', '했던', '▁것', '같', '고', '▁시설', '▁이용할', '▁수', '▁있는', '게', '▁한', '정', '적', '이', '여', '서', '▁아쉬움', '이', '▁', '컸', '음']
[[4497, 3, 2], [3758, 3, 2], [2962, 3, 2], [4045, 3, 2], [1723, 3, 2], [2959, 3, 2], [1638, 3, 2], [4977, 3, 2], [1024, 3, 2], [3647, 3, 2], [2149, 3, 2], [517, 7990, 3, 2], [5020, 3, 2], [905, 3, 2], [830, 3, 2], [993, 3, 2], [2981, 3, 2], [3729, 3, 2], [2872, 3, 2], [3860, 3, 2], [921, 3, 2], [4955, 3, 2], [4092, 3, 2], [3996, 3, 2], [3647, 3, 2], [3298, 3, 2], [2718, 3, 2], [3115, 3, 2], [3647, 3, 2], [3, 2], [517, 7522, 3, 2], [3606, 3, 2]]


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)


using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
from transformers import AutoModel, AutoTokenizer

kobert_tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1", use_fast=False)
kobert = AutoModel.from_pretrained("skt/kobert-base-v1")

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    

In [ ]:
train_dataloader

# Model 학습

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()  #평가모드

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/269 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.261980801820755 train acc 0.890625
epoch 1 batch id 201 loss 0.3883255422115326 train acc 0.8876710199004975
epoch 1 train acc 0.8865529006065349


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 test acc 0.8040277777777778


  0%|          | 0/269 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.17223000526428223 train acc 0.9375
epoch 10 batch id 201 loss 0.1733454316854477 train acc 0.941464552238806
epoch 10 train acc 0.9414467569947174


  0%|          | 0/90 [00:00<?, ?it/s]

epoch 10 test acc 0.8029861111111111


# 문장 테스트

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("불만족")
            elif np.argmax(logits) == 1:
                test_eval.append("만족")

        print(">> 입력 내용을 남긴 고객은 숙박기간동안 " + test_eval[0] + "하였습니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
end = 1
while end == 1 :
    sentence = input("Review를 입력해주세요. : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")


KeyboardInterrupt: ignored